In [1]:
# packages
import sys
import os
import math
import torch
import random
import pickle
import calendar
import numpy as np
import config
#import training 
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
datasets = os.listdir('./dataset')
datasets = [f for f in datasets if (f.startswith('Dataset') and f.endswith('.p'))]
datasets

['Dataset_acuteinflammation.p',
 'Dataset_balancescale.p',
 'Dataset_breastcancerwisc.p',
 'Dataset_cardiotocography3clases.p',
 'Dataset_energyy1.p',
 'Dataset_energyy2.p',
 'Dataset_iris.p',
 'Dataset_mammographic.p',
 'Dataset_pendigits.p',
 'Dataset_seeds.p',
 'Dataset_tictactoe.p',
 'Dataset_vertebralcolumn2clases.p',
 'Dataset_vertebralcolumn3clases.p']

In [3]:
for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]

    print(f'Dataset "{data_name}" has {N_feature} input features and {N_class} classes.\nThere are {N_train} training examples, {N_valid} valid examples, and {N_test} test examples in the dataset.')


Dataset "acuteinflammation" has 6 input features and 2 classes.
There are 70 training examples, 23 valid examples, and 25 test examples in the dataset.
Dataset "balancescale" has 4 input features and 3 classes.
There are 373 training examples, 124 valid examples, and 126 test examples in the dataset.
Dataset "breastcancerwisc" has 9 input features and 2 classes.
There are 418 training examples, 139 valid examples, and 140 test examples in the dataset.
Dataset "cardiotocography3clases" has 21 input features and 3 classes.
There are 1274 training examples, 424 valid examples, and 426 test examples in the dataset.
Dataset "energyy1" has 8 input features and 3 classes.
There are 459 training examples, 153 valid examples, and 154 test examples in the dataset.
Dataset "energyy2" has 8 input features and 3 classes.
There are 459 training examples, 153 valid examples, and 154 test examples in the dataset.
Dataset "iris" has 4 input features and 3 classes.
There are 88 training examples, 29 val

In [29]:
path = os.listdir('./results_different_strategy')
print(path)

['pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0220', 'pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0221', 'pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0222', 'pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0223', 'pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0224', 'pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0225', 'pNN_dataset\uf022acuteinflammation_learning_strategy\uf0220_training_strategy\uf0220_lr_theta\uf0220.1_lr_lnc\uf0220.0005_seed\uf0226', 'pNN_dataset\uf022acuteinflammation_lear

In [4]:
valid_acc = torch.zeros((13,3,2,6,10))
lr_theta = 0.1
lr_lncs = [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]
learning_strategys = [0, 1, 2] # 0 for neuron level, 1 for layer level, 2 for net level
training_strategys = [0, 1] # 0 for simutanous, 1 for alternative

for i in range(len(datasets)):
    dataset = datasets[i]
    datapath = os.path.join(f'./dataset/{dataset}')
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
    X_train    = data['X_train']
    y_train    = data['y_train']
    X_valid    = data['X_valid']
    y_valid    = data['y_valid']
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]

    print(f'Dataset "{data_name}" has {N_feature} input features and {N_class} classes.\nThere are {N_train} training examples, {N_valid} valid examples, and {N_test} test examples in the dataset.')

    # generate tensordataset
    trainset = TensorDataset(X_train, y_train)
    validset = TensorDataset(X_valid, y_valid)
    testset  = TensorDataset(X_test, y_test)

    # batch
    train_loader = DataLoader(trainset, batch_size=N_train)
    valid_loader = DataLoader(validset, batch_size=N_valid)
    test_loader  = DataLoader(testset,  batch_size=N_test)
    
    for j in range(len(learning_strategys)):
        learning_strategy = learning_strategys[j]
        
        for k in range(len(training_strategys)):
            training_strategy = training_strategys[k]
            
            for m in range(len(lr_lncs)):
                lr_lnc = lr_lncs[m]
                
    

                for n in range(10):
                    seed = n

                    #setup = f'dataset\uf022{data_name}_learning_strategy:{learning_strategy}_training_strategy:{training_strategy}_lr_theta:{lr_theta}_lr_lnc:{lr_lnc}_seed:{seed}'
                    setup = f'dataset\uf022{data_name}_learning_strategy\uf022{learning_strategy}_training_strategy\uf022{training_strategy}_lr_theta\uf022{lr_theta}_lr_lnc\uf022{lr_lnc}_seed\uf022{seed}'
                    
                    #print(setup)

                    #if os.path.exists(f'./results_different_strategy/pNN_{setup}'):
                    #print('File exists, pass.')
                    #else:
                    path = os.listdir('./results_different_strategy')
                    #print(path)
                    model = torch.load(f'./results_different_strategy/pNN_{setup}')
                    with torch.no_grad():
                        for x_valid, y_valid in valid_loader:
                            prediction = model(x_valid)
                            yhat_valid = torch.argmax(prediction.data, 1)
                            valid_correct = torch.sum(yhat_valid == y_valid.data)
                            acc_valid = valid_correct / y_valid.numel()
                            #print(acc_valid)
                            valid_acc[i,j,k,m,n] = acc_valid
            

Dataset "acuteinflammation" has 6 input features and 2 classes.
There are 70 training examples, 23 valid examples, and 25 test examples in the dataset.
Dataset "balancescale" has 4 input features and 3 classes.
There are 373 training examples, 124 valid examples, and 126 test examples in the dataset.
Dataset "breastcancerwisc" has 9 input features and 2 classes.
There are 418 training examples, 139 valid examples, and 140 test examples in the dataset.
Dataset "cardiotocography3clases" has 21 input features and 3 classes.
There are 1274 training examples, 424 valid examples, and 426 test examples in the dataset.
Dataset "energyy1" has 8 input features and 3 classes.
There are 459 training examples, 153 valid examples, and 154 test examples in the dataset.
Dataset "energyy2" has 8 input features and 3 classes.
There are 459 training examples, 153 valid examples, and 154 test examples in the dataset.
Dataset "iris" has 4 input features and 3 classes.
There are 88 training examples, 29 val

In [5]:
valid_acc

tensor([[[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [0.6087, 1.0000, 0.6087,  ..., 1.0000, 0.6087, 1.0000],
           [0.6087, 0.6087, 0.6087,  ..., 0.6087, 0.6087, 0.6087]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 0.7391, 1.0000,  ..., 0.9130, 1.0000, 1.0000],
           [0.6087, 0.7391, 0.6087,  ..., 0.6087, 0.6087, 0.6087]]],


         [[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000

## level

In [6]:
acc_level = valid_acc[:,:,0,0,:]
acc_level.shape, acc_level

(torch.Size([13, 3, 10]),
 tensor([[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
           1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
           1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
           1.0000, 1.0000]],
 
         [[1.0000, 0.9677, 1.0000, 1.0000, 1.0000, 0.9839, 0.9113, 0.9597,
           1.0000, 0.9597],
          [1.0000, 1.0000, 0.9597, 0.9435, 0.9919, 0.9274, 0.9839, 0.9274,
           0.9435, 0.9597],
          [0.9516, 0.9677, 0.9597, 0.9355, 0.9677, 0.9355, 1.0000, 0.9274,
           0.9839, 0.9516]],
 
         [[0.9640, 0.9640, 0.9640, 0.9784, 0.9712, 0.9640, 0.9640, 0.9640,
           0.9640, 0.9640],
          [0.9568, 0.9640, 0.9640, 0.9640, 0.9640, 0.9640, 0.9640, 0.9640,
           0.9640, 0.9712],
          [0.9640, 0.9568, 0.9640, 0.9568, 0.9640, 0.9568, 0.9640, 0.9640,
           0.9712, 0.9640]],
 
         [[0.8090, 0.8255, 0.8821, 0.8

In [7]:
acc_neuron = acc_level[:,0,:]
acc_layer = acc_level[:,1,:]
acc_net = acc_level[:,2,:]

In [11]:
neuron=acc_neuron.mean(dim=1)
neuron1=acc_neuron.std(dim=1)

In [12]:
layer = acc_layer.mean(dim=1)
layer1 = acc_layer.std(dim=1)

In [14]:
net = acc_net.mean(dim=1)
net1 = acc_net.std(dim=1)

## Table

In [15]:
for i in range(13):
    s = ""
    
    s = s+"& $" +str(neuron[i].numpy().round(3)) +"\pm "+str(neuron1[i].numpy().round(3)) +"$ "+"& $" +str(layer[i].numpy().round(3)) +"\pm "+str(layer1[i].numpy().round(3)) +"$ " + "& $" +str(net[i].numpy().round(3)) +"\pm "+str(net1[i].numpy().round(3))+"$"
        #s = str(datasets[i][:-2]) + s + "\\" + "\\"
    s = datasets[i][8:-2] + s + "\\" + "\\"
    print(s)
    print("\hline")

acuteinflammation& $1.0\pm 0.0$ & $1.0\pm 0.0$ & $1.0\pm 0.0$\\
\hline
balancescale& $0.978\pm 0.029$ & $0.964\pm 0.029$ & $0.958\pm 0.023$\\
\hline
breastcancerwisc& $0.966\pm 0.005$ & $0.964\pm 0.003$ & $0.963\pm 0.005$\\
\hline
cardiotocography3clases& $0.868\pm 0.041$ & $0.865\pm 0.062$ & $0.896\pm 0.017$\\
\hline
energyy1& $0.841\pm 0.182$ & $0.883\pm 0.04$ & $0.877\pm 0.041$\\
\hline
energyy2& $0.922\pm 0.03$ & $0.911\pm 0.061$ & $0.929\pm 0.007$\\
\hline
iris& $1.0\pm 0.0$ & $1.0\pm 0.0$ & $0.997\pm 0.011$\\
\hline
mammographic& $0.863\pm 0.007$ & $0.857\pm 0.022$ & $0.824\pm 0.091$\\
\hline
pendigits& $0.248\pm 0.091$ & $0.299\pm 0.177$ & $0.343\pm 0.144$\\
\hline
seeds& $0.917\pm 0.084$ & $0.939\pm 0.021$ & $0.946\pm 0.015$\\
\hline
tictactoe& $0.928\pm 0.063$ & $0.884\pm 0.095$ & $0.94\pm 0.063$\\
\hline
vertebralcolumn2clases& $0.848\pm 0.026$ & $0.836\pm 0.026$ & $0.836\pm 0.027$\\
\hline
vertebralcolumn3clases& $0.852\pm 0.026$ & $0.872\pm 0.019$ & $0.854\pm 0.016$\\
\hlin

In [16]:
neuron.mean(),layer.mean(),net.mean()

(tensor(0.8639), tensor(0.8672), tensor(0.8741))

In [17]:
neuron1.mean(),layer1.mean(),net1.mean()

(tensor(0.0449), tensor(0.0427), tensor(0.0354))

In [36]:
acc_seed = valid_acc.sum(4)/10
acc_seed.shape

torch.Size([13, 3, 2, 6])

In [37]:
acc_dataset = acc_seed.sum(0)/13

## compare

In [40]:
#rate
acc_rate= acc_dataset.sum(0).sum(0)/6
acc_rate#0.0005

tensor([0.8675, 0.8660, 0.8543, 0.8205, 0.6306, 0.5266])

In [41]:
#learn
acc_learn= acc_dataset.sum(1).sum(1)/12
acc_learn#net

tensor([0.7509, 0.7535, 0.7783])

In [42]:
#train
acc_train= acc_dataset.sum(0).sum(1)/18
acc_train#together

tensor([0.7709, 0.7509])

## analysis learn and rate

In [43]:
acc_train= acc_dataset.sum(1)/2

In [44]:
acc_train

tensor([[0.8625, 0.8679, 0.8468, 0.8042, 0.5926, 0.5314],
        [0.8694, 0.8647, 0.8589, 0.8210, 0.6014, 0.5059],
        [0.8708, 0.8656, 0.8572, 0.8362, 0.6977, 0.5424]])